In [5]:
from ngsolve import *
from netgen.geom2d import SplineGeometry

from netgen.occ import *
import netgen.meshing as ngm
import numpy as np
from ngsolve.webgui import *
from netgen.geom2d import unit_square

def innerL2(gfu,mesh):
    return Integrate(InnerProduct(gfu,gfu),mesh)
def L2error(gfu,exact,mesh):
    return sqrt(innerL2(gfu-exact,mesh))
geo = SplineGeometry()
geo.AddCircle(c=(0,0), r=2, bc="outer")
n = 3
mesh = Mesh(geo.GenerateMesh(maxh=0.5**n))


In [6]:
A= -0.34012
a = 0.34898 
u_exact = IfPos(sqrt(x**2 + y**2)-a, A*log(sqrt(x**2+y**2)),sqrt(0.25-((x)**2+(y)**2)))

# generate a triangular mesh of mesh-size 0.2
geo = SplineGeometry()
geo.AddCircle(c=(0,0), r=2)
n = 4
mesh = Mesh(geo.GenerateMesh(maxh=0.5**n))
phi = IfPos(0.5-sqrt(x**2+y**2), sqrt(0.25-((x)**2+(y)**2)), 0)

max_iter = 13
#phi = CF(sqrt(0.25-(x-0.5)**2-(y-0.5)**2) if (x-0.5)**2+(y-0.5)**2 < 0.25 else 0)
alpha = Parameter(2) # step size 
# psi initial guess
psik = GridFunction
# proximal galerkin iterations
fesu = H1(mesh, order=12, dirichlet="outer")
fesphi = L2(mesh, order=11) # L_inf 
fes = fesu*fesphi
 #bilinear form (not linear)
u, delta = fes.TrialFunction()
v, psi = fes.TestFunction()

# initial guesses
gfu_u = GridFunction(fesu)
gfu_u.Set(1)

psi = GridFunction(fesphi)
psi.Set(1)

uk = GridFunction(fesu)
psik = GridFunction(fesphi)
eps = 1e-6
max_iter_newton = 2
wh = GridFunction(fesu)
for k in range(1,max_iter):
    alpha.Set(2**k)
    print("Iteration of proximal galerkin: ", k)
    # assing uk, psik
    uk.vec.data = gfu_u.vec
    psik.vec.data = psi.vec
    print("Starting newton iteration")
    for i in range(max_iter_newton):
        wh.vec.data = gfu_u.vec
        # bilinear form
        a = BilinearForm(fes)
        a += alpha * grad(u) * grad(v)*dx + delta * v * dx
        a += u*psi*dx - (delta * exp(wh) * psi * dx + eps * (grad(delta) * grad(psi) * dx))
        # linear form rhs
        l = LinearForm(fes)
        # f = 0
        l += (alpha * 0 + psik-psi)*v*dx
        l += (phi + exp(psi))*psi*dx
        a.Assemble()
        l.Assemble()
        # solve the system
        gfu = GridFunction(fes)
        gfu.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs()) * l.vec
        gfu_u.vec.data = gfu.components[0].vec
        psi.vec.data = psi.vec + gfu.components[1].vec
    print("Exited newton iteration")
    Draw(gfu_u,mesh)





    



Iteration of proximal galerkin:  1
Starting newton iteration
Exited newton iteration


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

Iteration of proximal galerkin:  2
Starting newton iteration


KeyboardInterrupt: 

In [ ]:
geo = SplineGeometry()
geo.AddCircle(c=(0,0), r=2)
n = 3
mesh = Mesh(geo.GenerateMesh(maxh=0.5**n))
phi = IfPos(0.5-sqrt(x**2+y**2), sqrt(0.25-((x)**2+(y)**2)), 0)
Draw(phi, mesh, "tilde_u")

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [ ]:
# solve the simple laplacian example
fes = H1(mesh, order=2, dirichlet="bottom|right")
u, v = fes.TnT()
a = BilinearForm(fes)
a += grad(u)*grad(v)*dx
l = LinearForm(fes)
l += 0*v*dx
a.Assemble()
l.Assemble()
gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse(fes.FreeDofs()) * l.vec
Draw(gfu, mesh, "u")


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene